In [1]:
import math
import pickle
import numpy as np
import pandas as pd
import pickle as pklb
import datetime
from scipy.stats import beta
from tqdm import tqdm
from collections import defaultdict
import pickle as pkl

from transformers import BertTokenizer, BertModel

import os
import ast
#from Content import *
#from Venue import *

import torch

/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/u/yian3/.conda/envs/py39/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
## People
## Time
## Topic
## Media Contents


In [4]:
out_dir = '../data/aliceysu'
journalist = 'aliceysu'
with open(os.path.join(out_dir, f'{journalist}_dict.pkl'), 'rb') as f:
    data = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_ids.pkl'), 'rb') as f:
    map_id = pickle.load(f)
    
with open(os.path.join(out_dir, f'{journalist}_lan.pkl'), 'rb') as f:
    map_lan = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_type.pkl'), 'rb') as f:
    map_type = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_reply.pkl'), 'rb') as f:
    map_reply = pickle.load(f)

alice = pd.DataFrame.from_dict(data)
alice_sort = alice.sort_values(by=['created_at'])
conv = pd.read_csv(os.path.join(out_dir, f'{journalist}_conv_labels.csv'))

data = pkl.load(open(os.path.join(out_dir, f'{journalist}_dict.pkl'), 'rb'))
journal = pd.DataFrame.from_dict(data)
journal_sort = pd.read_csv(os.path.join(out_dir, f'{journalist}_conv_labels.csv'))

In [6]:
short_gap = []
for item in journal_sort['time gap']:
    if item < 240:
        short_gap.append(item)
        
max_gap = 240
data = []
# for index, item in journal_sort.iterrows():
#     data.append(superbeta.pdf(1-item['time gap'] / max_gap) + 1e-20)
for item in short_gap:
    data.append(superbeta.pdf(1 - item / max_gap) + 1e-10)

In [7]:
ids = list(set(journal_sort['conversation_id']))
batch_data = []
target_data = []
conv_data = []
ref_data = []
id_data = []
for idx in ids:
    convs = journal_sort[journal_sort['conversation_id'] == idx]
    convs_batch = convs[["type", "possibly_sensitive", "lang", "reply_settings",
                     "retweet_count", "reply_count", "like_count", "quote_count", "impression_count",
                     "mentions", "urls"]]
    conv_data.append(list(convs['conversation_id']))
    ref_data.append(list(convs['reference_id']))
    id_data.append(list(convs['tweet_id']))
    batch_data.append(convs_batch.values.tolist())
    target_data.append(list(convs['labels']))
    
label_data = target_data

In [258]:
contexts = []
for item in conv['context']:
    temp_s = ''
    if item == '0':
        contexts.append('Unified Twitter Taxonomy')

    else:
        for i in ast.literal_eval(item):
            temp_s  += (i['domain']['name'] + ' ' + i['entity']['name'])
        contexts.append(temp_s)
    
annotations = []
for item in conv['annotations']:
    temp_s = ''
    if item == '0':
        annotations.append('Others')

    else:
        for i in ast.literal_eval(item):
            temp_s += i['type'] + ' ' + i['normalized_text']

        annotations.append(temp_s)

In [9]:
journal_sort

,Unnamed: 0,author_id,conversation_id,user_id,reference_id,tweet_id,type,possibly_sensitive,lang,reply_settings,...,urls,labels,context,annotations,topics,ppls,topics_sim,ppls_sim,time gap,global
0,0,24709718,9947,339010742,9947,9948,0,0,1,0,...,0,1,Unified Twitter Taxonomy,Others,"['Unified Twitter Taxonomy', 'Ongoing News Sto...","['Others', 'Others']",[[0.7854743]],[[0.8374142]],1.0,"[[0, 0, 0], [1, 0, 0]]"
1,1,24709718,10491,805773481329704960,10491,10492,0,0,1,0,...,0,0,Unified Twitter Taxonomy,Other covid,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],1.0,"[[0, 0, 0], [1, 0, 0]]"
2,2,24709718,10020,14411162,10020,10021,0,0,1,0,...,0,1,Unified Twitter Taxonomy,Others,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],1.0,"[[0, 0, 0], [1, 0, 0]]"
3,3,964054011413483520,5887,3997957768,5894,5895,0,0,1,0,...,0,1,Place Israel,Others,"['Unified Twitter Taxonomy', 'Place Israel']","[""Unified Twitter Taxonomy NewsUnified Twitter...",[[0.86925256]],[[0.94740593]],1.0,"[[0, 0, 0], [1, 4, 0], [2, 0, 0]]"
4,4,990177925134630912,5887,3997957768,5892,5893,0,0,1,0,...,0,1,Place Israel,Others,"['Unified Twitter Taxonomy', 'Brand The Washin...","[""Unified Twitter Taxonomy NewsUnified Twitter...",[[0.8028154]],[[0.94740593]],0.0,"[[0, 0, 0], [1, 4, 1], [2, 0, 0]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10627,10627,24709718,10301,1598727471712456704,10301,10302,0,0,0,0,...,1,1,Unified Twitter Taxonomy,Others,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],528.0,"[[0, 0, 0], [1, 25, 25]]"
10628,10628,24709718,6208,1598727471712456704,6208,6210,0,0,0,0,...,1,1,Unified Twitter Taxonomy,Other Ultraman,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],105.0,"[[0, 0, 0], [1, 7, 6]]"
10629,10629,24709718,6208,1519143967886954496,6208,6209,0,0,0,0,...,1,0,Unified Twitter Taxonomy,Other UltramanOther UltramanPerson Xi Jinping,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],55.0,"[[0, 0, 0], [1, 7, 7]]"
10630,10630,24709718,0,1519143967886954496,0,1,0,0,0,0,...,1,1,Unified Twitter Taxonomy,Organization Jiangsu Consumer Protection Commi...,"['Unified Twitter Taxonomy', 'Unified Twitter ...","['Others', 'Others']",[[1.]],[[0.8374142]],298.0,"[[0, 0, 0], [1, 2, 2]]"


In [340]:
journal['context'] = contexts
journal['annotations'] = annotations
journal_sort = journal.sort_values(by=['created_at'])

In [341]:
## topics
## people
ref_ids = []

journal_sort['topics'] = None
journal_sort['ppls'] = None
conv_ids = set(journal_sort['conversation_id'])
for conv in list(conv_ids):
    test_d = journal_sort[journal_sort['conversation_id'] == conv]
    for index, item in test_d.iterrows():
        i = 0
        tweet_id = item['tweet_id']
        ref_id = item['reference_id']
        conv_id = item['conversation_id']
        topic1 = item['context']
        anno1 = item['annotations']
        if len(test_d) == 1:
            topic2 = 'Unified Twitter Taxonomy'
            anno2 = 'Others'
            journal_sort.at[index, 'topics']=[text1, text2]
            journal_sort.at[index, 'ppls']=[anno1, anno2]
            continue
        if ref_id not in ref_ids:
            if tweet_id == test_d.iloc[0]['tweet_id']:
                text2 = test_d[test_d['tweet_id']==test_d.iloc[1]['tweet_id']]['context'].item()
                anno2 = test_d[test_d['tweet_id']==test_d.iloc[1]['tweet_id']]['annotations'].item()
            else:
                text2 = test_d.iloc[i-1]['context']
                anno2 = test_d.iloc[i-1]['annotations']
        else:
            #print(tweet_id,ref_id)
            text2 = test_d[test_d['tweet_id']==ref_id]['context'].item()
            anno2 = test_d[test_d['tweet_id']==ref_id]['annotations'].item()

        ref_ids.append(tweet_id)
        
        journal_sort.at[index, 'topics']=[text1, text2]
        journal_sort.at[index, 'ppls']=[anno1, anno2]
        #print(text2, journal_sort.at[index, 'topics'])
        i += 1
        

In [57]:
journal_sort.to_csv(os.path.join(out_dir, f'{journalist}_context.csv'), index=False)

In [310]:
context_sims = []

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for index, item in tqdm(journal_sort.iterrows()):
    # Tokenize input text and convert to tensor
    c1 = item['topics'][0]
    c2 = item['topics'][1]
    
    a1 = item['ppls'][0]
    a2 = item['ppls'][1]
    inputs1 = tokenizer(c1, return_tensors="pt")
    inputs2 = tokenizer(c2, return_tensors="pt")

    # Generate embeddings
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # The last hidden state is the sequence of hidden states of the last layer of the model
    last_hidden_states1 = outputs1.last_hidden_state
    last_hidden_states2 = outputs2.last_hidden_state

    # Optionally, use the [CLS] token's embedding as the representation for the entire sentence
    sentence_embedding1 = last_hidden_states1[:, 0, :]
    sentence_embedding2 = last_hidden_states2[:, 0, :]

    cosine = np.dot(sentence_embedding1,sentence_embedding2.T)/(norm(sentence_embedding1)*norm(sentence_embedding2))
    context_sims.append(cosine)
    #print(sentence_embedding.shape)
    
journal_sort['topics_sim'] = context_sims

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
10632it [17:00, 10.41it/s]


In [346]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
anno_sims = []
for index, item in tqdm(journal_sort.iterrows()):
    # Tokenize input text and convert to tensor
    c1 = item['topics'][0]
    c2 = item['topics'][1]
    
    a1 = item['ppls'][0]
    a2 = item['ppls'][1]
    inputs1 = tokenizer(a1, return_tensors="pt")
    inputs2 = tokenizer(a2, return_tensors="pt")

    # Generate embeddings
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # The last hidden state is the sequence of hidden states of the last layer of the model
    last_hidden_states1 = outputs1.last_hidden_state
    last_hidden_states2 = outputs2.last_hidden_state

    # Optionally, use the [CLS] token's embedding as the representation for the entire sentence
    sentence_embedding1 = last_hidden_states1[:, 0, :]
    sentence_embedding2 = last_hidden_states2[:, 0, :]

    cosine = np.dot(sentence_embedding1,sentence_embedding2.T)/(norm(sentence_embedding1)*norm(sentence_embedding2))
    anno_sims.append(cosine)
    #print(sentence_embedding.shape)
    
journal_sort['ppls_sim'] = anno_sims
journal_sort = pd.read_csv(os.path.join(out_dir, f'{journalist}_context.csv'))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
10632it [13:57, 12.70it/s]


In [24]:
anchor_time = []
test_d = journal_sort[journal_sort['conversation_id']==4]
for index, item in test_d.iterrows():
    anchor_time.append(datetime.strptime(item['created_at'][:19], date_format))

In [25]:
anchor = anchor_time[1] - anchor_time[0]
anchor_hours = divmod(anchor.total_seconds() , 3600)[0]  # 1

In [26]:
## time
from datetime import datetime
ref_ids = []
date_format = "%Y-%m-%dT%H:%M:%S" 
journal_sort['time gap'] = None
conv_ids = set(journal_sort['conversation_id'])
for conv in list(conv_ids):
    test_d = journal_sort[journal_sort['conversation_id'] == conv]
    time0 = datetime.strptime(test_d.iloc[0]['created_at'][:19], date_format)
    k = 0
    for index, item in test_d.iterrows():
        tweet_id = item['tweet_id']
        ref_id = item['reference_id']
        conv_id = item['conversation_id']
        time1 = datetime.strptime(item['created_at'][:19], date_format)
        if len(test_d) == 1:
            journal_sort.at[index, 'time gap']= float(anchor_hours) #/120
            continue
        if ref_id not in ref_ids:
            if tweet_id == test_d.iloc[0]['tweet_id']:
                time2 = datetime.strptime(item['created_at'][:19], date_format) - anchor
            else:
                time2 = datetime.strptime(test_d.iloc[k-1]['created_at'][:19], date_format)
        else:
            #print(test_d[test_d['tweet_id']==ref_id]['created_at'][:19].item())
            time2 = datetime.strptime(test_d[test_d['tweet_id']==ref_id]['created_at'].item()[:19], date_format)
    

        ref_ids.append(tweet_id)
        gap_in_s = (time1 - time2).total_seconds() 
        journal_sort.at[index, 'time gap']= float(divmod(gap_in_s, 3600)[0]) #/ 120
        
        k += 1
        

In [ ]:
## PA * Latest * Field * Self = 2*2*2*2 = 16
## PA: Normal PA (p=0.1, p/#nodes + (1-p)indegree/sum_of_indegree), Uniform (1/#nodes)
## Latest: Normal Latest (beta(10,1), x=1-(outyear-inyear)/(outyear-oldest_year)), Uniform (1/(outyear-oldest_year))
## Field: Similar (1-(1-e^(-||x-y||_2))/(1-e^(-2)), x&y L2-normalized), Different ((1-e^(-||x-y||_2))/(1-e^(-2)), x&y L2-normalized)
## Self: Prefer (coauthors: 0.9/#coauthors, non-coauthors: 0.1/#non-coauthors), Not Prefer (coauthors: 0.1/#coauthors, non-coauthors: 0.9/#non-coauthors)

## 1st: Normal_PA * Normal_Latest * Similar_Field * Prefer_Self
## 2nd: Normal_PA * Normal_Latest * Similar_Field * NotPrefer_Self
## 3rd: Normal_PA * Normal_Latest * Different_Field * Prefer_Self
## 4th: Normal_PA * Normal_Latest * Different_Field * NotPrefer_Self
## 5th: Normal_PA * Uniform_Latest * Similar_Field * Prefer_Self
## 6th: Normal_PA * Uniform_Latest * Similar_Field * NotPrefer_Self
## 7th: Normal_PA * Uniform_Latest * Different_Field * Prefer_Self
## 8th: Normal_PA * Uniform_Latest * Different_Field * NotPrefer_Self
## 9th: Uniform_PA * Normal_Latest * Similar_Field * Prefer_Self
## 10th: Uniform_PA * Normal_Latest * Similar_Field * NotPrefer_Self
## 11th: Uniform_PA * Normal_Latest * Different_Field * Prefer_Self
## 12th: Uniform_PA * Normal_Latest * Different_Field * NotPrefer_Self
## 13th: Uniform_PA * Uniform_Latest * Similar_Field * Prefer_Self
## 14th: Uniform_PA * Uniform_Latest * Similar_Field * NotPrefer_Self
## 15th: Uniform_PA * Uniform_Latest * Different_Field * Prefer_Self
## 16th: Uniform_PA * Uniform_Latest * Different_Field * NotPrefer_Self

In [26]:
superbeta = beta(a=10,b=1)

# def cal_cite_edgeprobs(df):
#     edge_prob = []
#     for index, item in df.iterrows():
         
#         ppl_sim = float(item['ppls_sim'][2:-2])
#         ppl_dif = 1 - ppl_sim

#         topic_sim = float(item['topics_sim'][2:-2])
#         topic_dif = 1 - topic_sim

#         pnormal_latest = superbeta.pdf(1-item['time gap']) + 1e-20
#         puniform_latest = 1 / 168

#         temp1 = np.array([ppl_sim, ppl_dif])
#         temp2 = np.outer([topic_sim,topic_dif],[pnormal_latest,puniform_latest]).flatten()
#         result = np.outer(temp1,temp2).flatten()
#         edge_prob.append(result)
    
#     return np.array(edge_prob, dtype=np.float32)
def cal_cite_edgeprobs(ids):
    edge_prob = [] #np.zeros((len(ids), 8))
    for idx in ids:
        convs = journal_sort[journal_sort['conversation_id'] == idx]
        temp = []
        max_gap = 240 # max(convs['time gap'])
        for index, item in convs.iterrows():

            ppl_sim = float(item['ppls_sim'][2:-2])
            ppl_dif = 1 - ppl_sim

            topic_sim = float(item['topics_sim'][2:-2]) - 1e-3
            topic_dif = 1 - topic_sim

            pnormal_latest = (superbeta.pdf(1-item['time gap'] / max_gap) + 1e-10) / 10
            puniform_latest = 1 / 120

            temp1 = np.array([ppl_sim, ppl_dif])
            temp2 = np.outer([topic_sim,topic_dif],[pnormal_latest,puniform_latest]).flatten()
            result = np.outer(temp1,temp2).flatten()
            temp.append(result)
        edge_prob.append(np.array(temp))
    return edge_prob
edgeprob = cal_cite_edgeprobs(ids)

In [28]:
with open(os.path.join(out_dir, f'{journalist}_edgeprob.pkl'),'wb') as f:
    pickle.dump(edgeprob, f)

In [29]:
with open(os.path.join(out_dir, f'{journalist}_edgeprob.pkl'),'rb') as f:
    edgeprob = pickle.load(f)